In [1]:
import pandas as pd
import json
import requests
import pprint
from functools import reduce
from keys import fred_key, eia_key

In [2]:
# EIA API Calls

production = requests.get("https://api.eia.gov/series/?api_key="+ eia_key +"&series_id=PET.WCRFPUS2.W")
imports = requests.get("http://api.eia.gov/series/?api_key="+ eia_key +"&series_id=PET.WCRIMUS2.W")
supply = requests.get("http://api.eia.gov/series/?api_key="+ eia_key +"&series_id=PET.WRPUPUS2.W")

print(production.status_code)
print(imports.status_code)
print(supply.status_code)

200
200
200


In [3]:
# FRED API Calls
# The WTI returns price for the week ending Friday. This is the same release dat as the eia data

wti = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=WCOILWTICO&frequency=wef&api_key="+ 
                   fred_key +"&file_type=json")

# These indicators are only monthly

cpi = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=CPILFESL&frequency=m&api_key="+ 
                  fred_key +"&file_type=json")
unemployment = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=UNRATE&frequency=m&api_key="+ 
                  fred_key +"&file_type=json")
personal_consumption_expenditure = requests.get("https://api.stlouisfed.org/fred/series/observations?series_id=PCE&frequency=m&api_key="+ 
                  fred_key +"&file_type=json")

print(wti.status_code)
print(cpi.status_code)
print(unemployment.status_code)
print(personal_consumption_expenditure.status_code)

200
200
200
200


In [4]:
# Convert responses to JSONs

production_json = production.json()
imports_json = imports.json()
supply_json = supply.json()
wti_json = wti.json()
cpi_json = cpi.json()
unemp_json = unemployment.json()
pce_json = personal_consumption_expenditure.json()

In [5]:
# Extract EIA Data

production_series = production_json['series'][0]
imports_series = imports_json['series'][0]
supply_series = supply_json['series'][0]

In [6]:
production_df = pd.DataFrame(production_series['data'])
imports_df = pd.DataFrame(imports_series['data'])
supply_df = pd.DataFrame(supply_series['data'])

In [7]:
# Extract FRED Data

wti_observations = wti_json['observations']
Date = []
Value = []
for observation in wti_observations:
    Date.append(observation['date'])
    Value.append(observation['value'])
wti_df = pd.DataFrame(list(zip(Date,Value)))

In [8]:
cpi_observations = cpi_json['observations']
Date = []
Value = []
for observation in cpi_observations:
    Date.append(observation['date'])
    Value.append(observation['value'])
cpi_df = pd.DataFrame(list(zip(Date,Value)))

In [9]:
unemp_observations = unemp_json['observations']
Date = []
Value = []
for observation in unemp_observations:
    Date.append(observation['date'])
    Value.append(observation['value'])
unemp_df = pd.DataFrame(list(zip(Date,Value)))

In [10]:
pce_observations = pce_json['observations']
Date = []
Value = []
for observation in pce_observations:
    Date.append(observation['date'])
    Value.append(observation['value'])
pce_df = pd.DataFrame(list(zip(Date,Value)))

In [11]:
# Converting to datetime 

production_df.iloc[:,0] = pd.to_datetime(production_df.iloc[:,0],format='%Y%m%d', errors='raise')
imports_df.iloc[:,0] = pd.to_datetime(production_df.iloc[:,0],format='%Y%m%d', errors='raise')
supply_df.iloc[:,0] = pd.to_datetime(production_df.iloc[:,0],format='%Y%m%d', errors='raise')
wti_df.iloc[:,0] = pd.to_datetime(wti_df.iloc[:,0],format='%Y-%m-%d', errors='raise')
cpi_df.iloc[:,0] = pd.to_datetime(cpi_df.iloc[:,0],format='%Y-%m-%d', errors='raise')
unemp_df.iloc[:,0] = pd.to_datetime(unemp_df.iloc[:,0],format='%Y-%m-%d', errors='raise')
pce_df.iloc[:,0] = pd.to_datetime(pce_df.iloc[:,0],format='%Y-%m-%d', errors='raise')

In [12]:
# Rename columns

production_df = production_df.rename(columns = {0:'Date', 1:'Production (thousand barrels per day)'}).sort_values(by='Date').reset_index(drop=True)
imports_df = imports_df.rename(columns = {0:'Date', 1:'Imports (thousand barrels per day)'}).sort_values(by='Date').reset_index(drop=True)
supply_df = supply_df.rename(columns = {0:'Date', 1:'Supply (thousand barrels per day)'}).sort_values(by='Date').reset_index(drop=True)
wti_df = wti_df.rename(columns = {0:'Date', 1:'Price of Barrel (usd)'}).sort_values(by='Date')
cpi_df = cpi_df.rename(columns = {0:'Date', 1:'Core CPI (index 1982-1984=100)'}).sort_values(by='Date')
unemp_df = unemp_df.rename(columns = {0:'Date', 1:'Unemployment (%)'}).sort_values(by='Date')
pce_df = pce_df.rename(columns = {0:'Date', 1:'Personal Consumption Expenditure (billions of usd)'}).sort_values(by='Date')

In [13]:
# Merge EIA data and WTI

data_frames = [supply_df,imports_df,production_df,wti_df]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='left'), data_frames)

In [14]:
# Merge FRED data

data_frames = [cpi_df, pce_df, unemp_df]
indicators_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='left'), data_frames)

In [15]:
# Creating new datetime columns to merge on

indicators_merged['Merge Date']= indicators_merged['Date'].dt.strftime('%m-%Y')

In [16]:
df_merged['Merge Date'] = df_merged['Date'].dt.strftime('%m-%Y')

In [17]:
# Merging and cleaning columns

final_df = pd.merge(df_merged, indicators_merged, how='left', on='Merge Date').fillna(method='ffill')
final_df = final_df[['Date_x', 'Merge Date', 'Personal Consumption Expenditure (billions of usd)',
                     'Core CPI (index 1982-1984=100)', 'Unemployment (%)', 'Supply (thousand barrels per day)', 
                     'Imports (thousand barrels per day)', 'Production (thousand barrels per day)', 'Price of Barrel (usd)']]
final_df = final_df.rename(columns = {"Date_x":"Date", "Merge Date":"Month"})
final_df.iloc[-1,:].values
final_df

,Date,Month,Personal Consumption Expenditure (billions of usd),Core CPI (index 1982-1984=100),Unemployment (%),Supply (thousand barrels per day),Imports (thousand barrels per day),Production (thousand barrels per day),Price of Barrel (usd)
0,1990-11-16,11-1990,3871.9,138.0,6.2,16588,5637,6910,31.50
1,1990-11-23,11-1990,3871.9,138.0,6.2,17019,5610,7440,30.69
2,1990-11-30,11-1990,3871.9,138.0,6.2,15686,4532,7235,32.32
3,1990-12-07,12-1990,3861.3,138.6,6.3,17753,5007,6996,27.72
4,1990-12-14,12-1990,3861.3,138.6,6.3,16901,5236,7474,26.39
5,1990-12-21,12-1990,3861.3,138.6,6.3,16669,5445,7245,27.56
6,1990-12-28,12-1990,3861.3,138.6,6.3,17953,4468,7559,27.21
7,1991-01-04,01-1991,3841.0,139.5,6.4,17319,5536,7481,26.38
8,1991-01-11,01-1991,3841.0,139.5,6.4,15968,4866,7475,27.55
9,1991-01-18,01-1991,3841.0,139.5,6.4,17171,5053,7461,26.85


In [18]:
# Staggering the price of WTI. This is so our independent variables are actually trained to predict next week's price.

PriceOfBarrel = final_df["Price of Barrel (usd)"]
StaggeredList=[]
count=1
while(count<len(PriceOfBarrel)):
    StaggeredList.append(PriceOfBarrel[count])
    count +=1
StaggeredList.append(0)
modeling_df = final_df.copy()
modeling_df ["Staggered Price of Barrel"] = StaggeredList
modeling_df = modeling_df.iloc[:-1]

modeling_df['Personal Consumption Expenditure (billions of usd)'] = pd.to_numeric(modeling_df['Personal Consumption Expenditure (billions of usd)'],errors='coerce')
modeling_df['Core CPI (index 1982-1984=100)'] = pd.to_numeric(modeling_df['Core CPI (index 1982-1984=100)'],errors='coerce')
modeling_df['Unemployment (%)'] = pd.to_numeric(modeling_df['Unemployment (%)'],errors='coerce')
modeling_df['Price of Barrel (usd)'] = pd.to_numeric(modeling_df['Price of Barrel (usd)'],errors='coerce')
modeling_df['Staggered Price of Barrel'] = pd.to_numeric(modeling_df['Staggered Price of Barrel'],errors='coerce')

modeling_df.to_csv("Data.csv")

In [20]:
# Linear Regression Model Without Scaling

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

model = LinearRegression()

X = modeling_df[["Personal Consumption Expenditure (billions of usd)",
              "Unemployment (%)",
              "Supply (thousand barrels per day)",
              "Imports (thousand barrels per day)",
              "Production (thousand barrels per day)"]]
y = modeling_df["Staggered Price of Barrel"].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

model.fit(X_train, y_train)

training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.7466429286323502
Testing Score: 0.7367656545261481


In [21]:
# Scaling data

from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [22]:
# Decision Tree Model

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

dtm = DecisionTreeRegressor(max_depth=4,
                           min_samples_split=5,
                            max_leaf_nodes=10)

dtm.fit(X_train_scaled, y_train)

training_score = dtm.score(X_train_scaled, y_train)
testing_score = dtm.score(X_test_scaled, y_test)
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.9330729950624042
Testing Score: 0.9374077817878883


In [23]:
# Random Forest Model

rfm = RandomForestRegressor(n_estimators=1000)

rfm.fit(X_train_scaled, y_train.ravel())

training_score = rfm.score(X_train_scaled, y_train)
testing_score = rfm.score(X_test_scaled, y_test)
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.9982380624276718
Testing Score: 0.989873932871208


In [24]:
y_predictions = rfm.predict(X_test_scaled)
mean_absolute_error(y_test, y_predictions)

2.0837265274151346

In [25]:
sorted(zip(rfm.feature_importances_, X), reverse=True)

[(0.8748332017506206, 'Personal Consumption Expenditure (billions of usd)'),
 (0.07805769612430319, 'Unemployment (%)'),
 (0.0362212403440362, 'Production (thousand barrels per day)'),
 (0.005607444793715673, 'Supply (thousand barrels per day)'),
 (0.0052804169873244, 'Imports (thousand barrels per day)')]

In [26]:
from dateutil.relativedelta import relativedelta
most_recent_date = (final_df.iloc[-1,0]).date()
next_week_date = most_recent_date + relativedelta(weeks=+1)

In [39]:
most_recent_price = str(final_df.iloc[-1,8])

'26.09'

In [41]:
X = final_df[["Personal Consumption Expenditure (billions of usd)",
              "Unemployment (%)",
              "Supply (thousand barrels per day)",
              "Imports (thousand barrels per day)",
              "Production (thousand barrels per day)"]]

all_predictions = rfm.predict(X_scaler.transform(X)).round(2)

In [45]:
all_predictions = all_predictions.tolist()
date = final_df['Date'].tolist()
oil_price = final_df['Price of Barrel (usd)'].tolist()

In [46]:
price_next_week = all_predictions[-1]
price_next_week

51.07

In [47]:
print("The price of oil (WTI) on Friday " + str(most_recent_date) + " was $" + str(most_recent_price) 
      +". Our model predicts that the price of oil on Friday " + str(next_week_date) + " will be $" + str(price_next_week) + ".")

The price of oil (WTI) on Friday 2020-03-20 was $26.09. Our model predicts that the price of oil on Friday 2020-03-27 will be $51.07.


In [51]:
predictions_df = pd.DataFrame(list(zip(date, all_predictions, oil_price)), columns=['Date','Predictions','Actual Price'])
predictions_df.head()

,Date,Predictions,Actual Price
0,1990-11-16,29.91,31.50
1,1990-11-23,30.48,30.69
2,1990-11-30,27.93,32.32
3,1990-12-07,26.20,27.72
4,1990-12-14,27.19,26.39


In [63]:
Predictions_UnScattered = [31.50]
wrong_predictions = predictions_df["Predictions"]
wrong_predictions
count = 0
while count < len(wrong_predictions) - 1:
    Predictions_UnScattered.append(wrong_predictions[count])
    count = count + 1
predictions_df = pd.DataFrame(list(zip(date, Predictions_UnScattered, oil_price)), columns=['Date','Predictions','Actual Price'])
predictions_df.tail()

,Date,Predictions,Actual Price
1527,2020-02-21,47.98,53.14
1528,2020-02-28,50.69,48.36
1529,2020-03-06,49.48,45.57
1530,2020-03-13,41.73,32.39
1531,2020-03-20,39.61,26.09


In [61]:
import plotly.graph_objects as go

Predictions = predictions_df["Predictions"]
actual_price = predictions_df['Actual Price']
Date = predictions_df['Date']
fig = go.Figure()
fig.add_trace(go.Scatter(x=Date, y=Predictions,
             mode='lines+markers',
             name= 'Predicted',
                         marker_color='Darkred'))
fig.add_trace(go.Scatter(x=Date, y=actual_price,
                        mode='lines',
                        name='Actual Price',
                        marker_color="Green"))
fig.update_layout(title='Price of Oil vs. Projections', xaxis_title = 'Date', yaxis_title = 'USD/Barrel')
fig.show()

In [60]:
# Plot Scatter 
PriceOfBarrel = final_df["Price of Barrel (usd)"]

Date = final_df["Date"]
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=Date, y=PriceOfBarrel,
                    mode='lines',
                    name='lines',
                        marker_color='Darkred'))

fig.update_layout(title='Price of Oil', xaxis_title = 'Date', yaxis_title = 'USD/Barrel')

fig.show()

In [36]:
# Plot Scatter
predictions = model.predict(X)
predictions_x = [ item for elem in predictions for item in elem]
 
residuals = predictions - y
residuals_y = [ item for elem in residuals for item in elem]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=predictions_x, y=residuals_y,
                    mode='markers',
                    name='predictions',
                        marker_color='LightSkyBlue'))

# adapt this line to make it work with plotly= plt.hlines(y=0, xmin=predictions.min(), xmax=predictions.max())
fig.add_trace(go.Scatter(x=[min(predictions_x), max(predictions_x)],y=[0]* len(predictions_x) ,
                    mode='lines+markers',
                    name='lines+markers'))

fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.update_layout(title='Scatter, Predictions vs Residuals',
                 xaxis_title = "Predictions",
                  yaxis_title = "Residuals"
                 )

fig.show()

In [37]:
# Plot Scatter
predictions = model.predict(X_train)
predictions_x_train = [ item for elem in predictions for item in elem]
 
residuals = predictions - y_train
residuals_y_train = [ item for elem in residuals for item in elem]

predictions = model.predict(X_test)
predictions_x_test = [ item for elem in predictions for item in elem]
 
residuals = predictions - y_test
residuals_y_test = [ item for elem in residuals for item in elem]

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=predictions_x_train, y=residuals_y_train,
                    mode='markers',
                    name='Train_data',
                        marker_color='LightSkyBlue'))

fig.add_trace(go.Scatter(x=predictions_x_test, y=residuals_y_test,
                    mode='markers',
                    name='Test_data',
                        marker_color='MediumPurple'))

# adapt this line to make it work with plotly= plt.hlines(y=0, xmin=predictions.min(), xmax=predictions.max())
fig.add_trace(go.Scatter(x=[min(predictions_x_train), max(predictions_x_train)],y=[0]* len(predictions_x_train) ,
                    mode='lines+markers',
                    name='lines+markers'
                        ))

fig.update_traces(marker=dict(size=12,
                              line=dict(width=1.25,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.update_layout(title='Scatter, Train vs. Test',xaxis_title = "Predictions",
                  yaxis_title = "Residuals")

fig.show()